In [4]:
import nltk
import pandas as pd
import math
from nltk.corpus import brown
import numpy as np

In [5]:
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
# I found that string.punctuation did not include all the punctuations of brown words, here I added extra punctuation in the following list
punctuation = ['!','"','#','$','%','&',"'",'(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~','``',"''",'--']
lower_words = [x.lower() for x in brown.words()]
pun_stop = punctuation + stopwords.words('english')
filter_words1 = [x for x in lower_words if x not in pun_stop]
filter_words = list(filter(lambda x: x.isalpha() and len(x) > 1, filter_words1)) # remove numbers and single letter words

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/encrypted_soul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import collections
import operator
words_count = dict(collections.Counter(filter_words))
sorted_words = sorted(words_count.items(), key = operator.itemgetter(1), reverse = True)
# first 5000 most commonly-occuring words
V = [x[0] for x in sorted_words]
C = V

In [5]:
import itertools
def ls_uniq(seq):
   checked = []
   for e in seq:
       if e not in checked:
           checked.append(e)
   return checked
c_words = []
for v_word in V:
    four_words = []
    positions = [x for x, n in enumerate(filter_words) if n == v_word] # locate each word of V in filter_words
    for i in positions:
        if i ==0:
            four_word = filter_words[1:3]
        elif i == 1:
            four_word = ([filter_words[0]] + filter_words[2:4])
        else:
            four_word = (filter_words[(i-2):i] + filter_words[(i+1):(i+3)])
        four_word_uniq = ls_uniq(four_word)
        four_words = four_words + four_word_uniq
    four_words_count = dict(collections.Counter(four_words))
    window_count = len(positions)
    for c_word in four_words_count:
        if c_word in C:
            cword_fre = four_words_count[c_word]
            Pr_cw = cword_fre/window_count
            c_words.append((v_word, c_word, cword_fre, window_count, Pr_cw))
cwords = pd.DataFrame(c_words)
cwords.columns = ['V_Word','C_Word','Cword_Count','Window_Count','Pr_cw']
cwords.head()

,V_Word,C_Word,Cword_Count,Window_Count,Pr_cw
0,one,foster,2,3292,0.000608
1,one,homes,2,3292,0.000608
2,one,major,12,3292,0.003645
3,one,items,2,3292,0.000608
4,one,atlanta,2,3292,0.000608


In [6]:
cwords_uniq = list(cwords['C_Word'].unique())
cwords_pro = {}
for cword in cwords_uniq:
    cwords_pro[cword] = filter_words.count(cword) / len(filter_words)
def cword_pro(x):
    return cwords_pro[x]
cwords['Pr_c'] = cwords['C_Word'].apply(cword_pro)
cwords.head()

,V_Word,C_Word,Cword_Count,Window_Count,Pr_cw,Pr_c
0,one,foster,2,3292,0.000608,0.000029
1,one,homes,2,3292,0.000608,0.000122
2,one,major,12,3292,0.003645,0.000486
3,one,items,2,3292,0.000608,0.000142
4,one,atlanta,2,3292,0.000608,0.000069


In [7]:
def max_log(row):
    f = row['Pr_cw']
    g = row['Pr_c']
    l = math.log(f/g)
    return max(0, l)
cwords['f_w'] = cwords.apply(max_log, axis = 1)
cwords.head()

,V_Word,C_Word,Cword_Count,Window_Count,Pr_cw,Pr_c,f_w
0,one,foster,2,3292,0.000608,0.000029,3.025324
1,one,homes,2,3292,0.000608,0.000122,1.606240
2,one,major,12,3292,0.003645,0.000486,2.015746
3,one,items,2,3292,0.000608,0.000142,1.456709
4,one,atlanta,2,3292,0.000608,0.000069,2.178027


In [8]:
mutal_words = pd.pivot_table(cwords, index = 'V_Word', columns = 'C_Word', values = 'f_w')
mutal_words.head()

C_Word,aa,aaa,aaawww,aah,aaron,ab,aback,abandon,abandoned,abandoning,...,zoooop,zorrillas,zounds,zu,zubkovskaya,zur,zurcher,zurich,zwei,zworykin
V_Word,,,,,,,,,,,,,,,,,,,,,
aa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aaa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aaawww,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aaron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
mutal_words = mutal_words.fillna(0)
mutal_words.head()

C_Word,aa,aaa,aaawww,aah,aaron,ab,aback,abandon,abandoned,abandoning,...,zoooop,zorrillas,zounds,zu,zubkovskaya,zur,zurcher,zurich,zwei,zworykin
V_Word,,,,,,,,,,,,,,,,,,,,,
aa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaawww,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaron,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.decomposition import TruncatedSVD
# from sklearn.random_projection import sparse_random_matrix
X = np.asarray(mutal_words)
svd = TruncatedSVD(n_components = 100, n_iter = 7, random_state = 42)
svd.fit(X)
X_reduce = svd.fit_transform(X)
X_reduce_df = pd.DataFrame(X_reduce, index = mutal_words.index)
X_reduce_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
V_Word,,,,,,,,,,,,,,,,,,,,,
aa,1.239446,0.300440,-0.125250,0.313736,-2.422761,0.170843,0.643527,-0.286860,1.476253,0.102527,...,-0.069631,-0.276605,0.253560,0.172611,0.008375,0.297959,0.035520,-0.332405,-0.800640,-0.351296
aaa,0.550404,0.205692,0.250945,-0.336509,-0.417510,-0.057363,0.000530,0.050449,-0.109208,-0.191278,...,-0.896207,0.182914,-0.023720,0.500662,0.879271,0.087068,0.059997,-0.061481,-0.149309,0.367078
aaawww,0.693837,0.939585,-0.040964,0.012556,-0.405223,0.632578,-0.168105,-0.330152,-0.287677,-0.171594,...,0.030048,0.288327,0.228329,0.111810,0.366543,0.144393,-0.271948,-0.545452,0.089754,0.419006
aah,0.834294,0.897204,-0.538317,0.509594,-1.325316,1.018558,0.749526,0.339040,-0.279959,-0.366885,...,0.149727,-0.409634,-0.161757,-0.328594,0.113039,-0.160558,-0.547223,-0.014393,0.217700,0.055211
aaron,2.060484,1.170646,-0.330354,0.500875,-0.448220,-0.282321,0.027295,-0.071805,0.038457,-0.770725,...,-0.160163,-0.359900,0.051960,0.454907,-0.004977,-0.742583,-1.318023,0.271884,-0.485214,-0.023480


In [11]:
from sklearn.metrics.pairwise import cosine_similarity as cs
words_similarity = 1 - cs(X_reduce, X_reduce)
words_similarity_df = pd.DataFrame(words_similarity, index =
mutal_words.index,columns = mutal_words.index)
np.fill_diagonal(words_similarity_df.values, 1)
words_similarity_df.head()

/Users/encrypted_soul/.virtualenvs/blaze--experiments/lib/python3.9/site-packages/sklearn/utils/extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


V_Word,aa,aaa,aaawww,aah,aaron,ab,aback,abandon,abandoned,abandoning,...,zoooop,zorrillas,zounds,zu,zubkovskaya,zur,zurcher,zurich,zwei,zworykin
V_Word,,,,,,,,,,,,,,,,,,,,,
aa,1.000000,0.975008,1.011654,0.944812,1.004668,1.032521,0.937451,0.736088,0.791499,0.888558,...,0.977647,1.072575,1.012604,1.074264,0.924276,1.062689,0.880737,0.964717,0.980203,1.095959
aaa,0.975008,1.000000,0.938052,0.978055,0.937789,1.012175,0.819395,0.879316,0.917536,0.955100,...,0.917650,1.114090,1.004578,0.945908,0.969840,0.978610,1.077980,1.025556,0.999779,0.819805
aaawww,1.011654,0.938052,1.000000,0.681455,0.750467,0.937263,0.845970,1.066881,1.002549,1.003718,...,0.888096,0.920022,0.857682,0.868310,0.989729,1.087347,1.121745,0.906732,1.059431,0.877826
aah,0.944812,0.978055,0.681455,1.000000,0.859028,1.006494,0.863532,1.046024,0.945937,0.632273,...,0.887069,0.941485,0.873539,0.986404,1.019546,1.043933,0.948647,0.460929,1.019525,0.931749
aaron,1.004668,0.937789,0.750467,0.859028,1.000000,0.892651,0.717558,0.824302,0.799864,0.782923,...,0.856734,0.942761,0.861250,0.953786,1.032697,1.010009,0.800045,0.913998,1.089562,0.939829


In [20]:
import pandas as pd

# Load SimLex-999
simlex_path = 'human_annoted_scores/SimLex-999/SimLex-999.txt'
simlex_data =pd.read_csv(simlex_path, sep='\t')

In [21]:
simlex_data.head()

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93


In [24]:
similarity_scores = []

for _, row in simlex_data.iterrows():
    print(row)
    word1 = row['word1']
    word2 = row['word2']
    if word1 in words_similarity_df.columns and word2 in words_similarity_df.columns:
        score = words_similarity_df[word1][word2]
        similarity_scores.append({'word1': word1, 'word2': word2, 'model_score': score, 'simlex_score': row['SimLex999']})

similarity_scores_df = pd.DataFrame(similarity_scores)

word1           old
word2           new
POS               A
SimLex999      1.58
conc(w1)       2.72
conc(w2)       2.81
concQ             2
Assoc(USF)     7.25
SimAssoc333       1
SD(SimLex)     0.41
Name: 0, dtype: object
word1                smart
word2          intelligent
POS                      A
SimLex999              9.2
conc(w1)              1.75
conc(w2)              2.46
concQ                    1
Assoc(USF)            7.11
SimAssoc333              1
SD(SimLex)            0.67
Name: 1, dtype: object
word1               hard
word2          difficult
POS                    A
SimLex999           8.77
conc(w1)            3.76
conc(w2)            2.21
concQ                  2
Assoc(USF)          5.94
SimAssoc333            1
SD(SimLex)          1.19
Name: 2, dtype: object
word1             happy
word2          cheerful
POS                   A
SimLex999          9.55
conc(w1)           2.56
conc(w2)           2.34
concQ                 1
Assoc(USF)         5.85
SimAssoc333        

In [25]:
from scipy.stats import spearmanr

spearman_corr, _ = spearmanr(similarity_scores_df['model_score'], similarity_scores_df['simlex_score'])
print(f"Spearman's rank correlation coefficient: {spearman_corr}")

Spearman's rank correlation coefficient: -0.045564976642775694
